In [2]:
import random
import sys
# We need this import to connect to the MySQL database
import mysql.connector

# --- 1. GAME CONSTANTS AND INITIAL STATE ---
MAX_TIME_MINUTES = 1440 # Total time limit for the mission
START_HEALTH = 75.0
HEALING_TIME_BASE = 60

current_health = START_HEALTH
total_time_minutes = 0
current_location_icao = None
target_hospital_icao = None

# Variables to hold all game data
airports = {}
interconnections = []
risks = []

# ----------------------------------------------------------------------
# Helper Function for User Input
# ----------------------------------------------------------------------
def GET_USER_RESPONSE(prompt):
    """Reads the player's action from the console."""
    user_input = input(prompt)
    # .strip().upper() cleans the input for easy comparison
    return user_input.strip().upper()

# ----------------------------------------------------------------------
# Data Loading Functions (Focus on TRY/EXCEPT and INNER JOIN)
# ----------------------------------------------------------------------

def _load_hardcoded_fallback():
    """Loads simple data if the database connection fails."""
    global airports, interconnections, risks

    # 1. Airports Data
    airports.update({
        'OTHH': {'Name': 'Hamad Intl', 'Continent': 'Asia', 'Clinic': True, 'Healing': 25.0, 'TimeFactor': 0.75},
        'EGLL': {'Name': 'London Heathrow', 'Continent': 'Europe', 'Clinic': True, 'Healing': 20.0, 'TimeFactor': 0.50},
        'KJFK': {'Name': 'JFK Airport', 'Continent': 'N. America', 'Clinic': False},
    })

    # 2. Flight Connections
    interconnections.extend([
        {'Departure_Airport_ID': 'OTHH', 'Arrival_Airport_ID': 'EGLL', 'Time': 420, 'Health_Cost_Per_Minute': 0.045},
        {'Departure_Airport_ID': 'EGLL', 'Arrival_Airport_ID': 'OTHH', 'Time': 400, 'Health_Cost_Per_Minute': 0.050},
        {'Departure_Airport_ID': 'OTHH', 'Arrival_Airport_ID': 'KJFK', 'Time': 800, 'Health_Cost_Per_Minute': 0.035},
    ])

    # 3. Risk Data (Simplified)
    risks.extend([
        {'Name': 'Engine Trouble', 'Probability': 0.50, 'TimePenalty': 90, 'HealthPenalty': 6.00},
    ])

    print("WARNING: Database connection failed. Loaded FALLBACK (hardcoded) data instead.")


def load_game_data():
    """
    Attempts to load data from the database using a safe try/except block.
    If the connection fails, it calls the hardcoded fallback function.
    """
    global airports, interconnections

    connection = None
    cursor = None

    # --- START TRY/EXCEPT BLOCK for Database Connection ---
    try:
        # Database connection details (replace these with your actual credentials if needed)
        connection = mysql.connector.connect(
            host='mysql.metropolia.fi',
            port=3306,
            database='sadhanid',
            user='sadhanid',
            password='123456',
            autocommit=True
        )
        cursor = connection.cursor(dictionary=True)
        print("--- Database connection successful. Loading data... ---")

        # SQL Query with INNER JOIN
        # This joins the 'Airport' table with the 'Continent' table to get the full location name
        airport_sql_query = """
                            SELECT A.ICAO_Code,
                                   A.Airport_Name,
                                   C.Continent_Name,
                                   A.Clinic,
                                   A.Clinic_Healing_Amount,
                                   A.Clinic_Time_Factor
                            FROM Airport AS A
                            INNER JOIN Continent AS C
                                ON A.Continent_ID = C.Continent_ID
                            """
        cursor.execute(airport_sql_query)

        # Load Airport Data
        for row in cursor.fetchall():
            icao_code = row['ICAO_Code']
            is_clinic = bool(row['Clinic'])
            airports[icao_code] = {
                'Name': row['Airport_Name'],
                'Continent': row.get('Continent_Name', 'Unknown'),
                'Clinic': is_clinic,
                'Healing': float(row['Clinic_Healing_Amount']) if is_clinic else 0.0,
                'TimeFactor': float(row['Clinic_Time_Factor']) if is_clinic else 0.0,
            }

        # In a real game, you would load flights and risks here too...

        print(f"Successfully loaded {len(airports)} airports from the database.")

    except mysql.connector.Error as errors:
        # --- EXECUTES IF CONNECTION FAILS ---
        print(f"Database Error occurred: {errors}")
        _load_hardcoded_fallback()

    finally:
        # --- Executes whether or not an error occurred (ALWAYS close resources) ---
        if cursor:
            cursor.close()
        if connection and connection.is_connected():
            connection.close()
    # --- END TRY/EXCEPT BLOCK ---


# ----------------------------------------------------------------------
# Game Flow and Logic Functions (Focus on RETURN)
# ----------------------------------------------------------------------

def initialize_game():
    """Sets up the initial location, target, and health/time."""
    global current_health, total_time_minutes, current_location_icao, target_hospital_icao

    load_game_data()

    if not airports:
        # If no data was loaded (even the fallback failed), exit
        print("FATAL ERROR: Could not load any airport data.")
        sys.exit()

    current_health = START_HEALTH
    total_time_minutes = 0

    all_icaos = list(airports.keys())

    # Set the starting location and choose a random target
    current_location_icao = random.choice(all_icaos)
    icao_remaining = [icao for icao in all_icaos if icao != current_location_icao]

    if not icao_remaining:
        target_hospital_icao = current_location_icao
    else:
        target_hospital_icao = random.choice(icao_remaining)


    print(f"\n------ BEGIN MISSION ({START_HEALTH} HP) ------")
    print(f"STARTING AT: {airports[current_location_icao]['Name']}.")
    print(f"GOAL: Reach {airports[target_hospital_icao]['Name']}.")


def check_game_over():
    """Checks if the player has won or lost. Uses return True/False."""
    global current_health, total_time_minutes, current_location_icao, target_hospital_icao

    if current_health <= 0:
        print("\n😭 MISSION FAILED: Patient lost (Health dropped to 0).")
        # return True signals that the game is over
        return True
    elif total_time_minutes >= MAX_TIME_MINUTES:
        print("\n😭 MISSION FAILED: Time limit exceeded.")
        return True
    elif current_location_icao == target_hospital_icao and current_health > 0:
        print("\n🏆 MISSION SUCCESS: Patient delivered!")
        return True
    else:
        # return False signals that the game must continue
        return False


def _execute_flight(flight_information):
    """Applies flight costs and updates the location."""
    global current_health, total_time_minutes, current_location_icao

    time_cost = flight_information['Time']
    health_cost = flight_information['Health_Loss']

    current_health -= health_cost
    total_time_minutes += time_cost

    # Check for game over *before* updating location in case the patient is lost mid-flight
    if check_game_over():
        return True # Game over happened!

    current_location_icao = flight_information['Dest_ICAO']
    destination_airport = airports[current_location_icao]

    print(f"--- ARRIVAL: Arrived at {destination_airport['Name']}. Time: -{time_cost} min. Health: -{health_cost:.2f} HP. ---")

    # The function returns the result of the game over check (True/False)
    return check_game_over()


def apply_risk_check(flight_info):
    """Checks for a single risk event and applies penalty."""
    global current_health, total_time_minutes

    # Simplified check: Only one risk type for basic learners
    if risks and random.random() < risks[0]['Probability']:
        risk = risks[0]
        print(f"\n⚠️ EMERGENCY: {risk['Name']}!")

        total_time_minutes += risk['TimePenalty']
        current_health -= risk['HealthPenalty']
        print(f"PENALTY: Time +{risk['TimePenalty']} min, Health -{risk['HealthPenalty']:.2f} HP.")

        if check_game_over():
            # return True if a game-ending condition was reached
            return True

        # Simple input for a basic choice
        while True:
            choice = GET_USER_RESPONSE("Proceed (P) or Cancel (C) mission? ")
            if choice == 'C':
                print("🛑 Flight cancelled. Returning to airport action phase.")
                return "CANCELLED" # Custom return value to signal cancellation
            elif choice == 'P':
                break
            else:
                print("❌ Invalid input. Enter 'P' or 'C'.")

    # If no fatal risk occurred and the flight proceeds, return False
    return False


def handle_player_turn():
    """Handles the main decision loop for the player."""

    available_flights = []
    flight_id = 1
    # Find available flights from current location
    for connection in interconnections:
        if connection['Departure_Airport_ID'] == current_location_icao:
            time = connection['Time']
            health_cost = time * connection['Health_Cost_Per_Minute']
            available_flights.append({
                'ID': flight_id,
                'Dest_ICAO': connection['Arrival_Airport_ID'],
                'Time': time,
                'Health_Loss': health_cost,
            })
            flight_id += 1

    print("\n--- YOUR MOVE ---")
    for flight in available_flights:
        dest_airport = airports.get(flight['Dest_ICAO'])
        print(f"  [{flight['ID']}] Fly to {dest_airport['Name']} (Loss: {flight['Health_Loss']:.2f} HP)")

    print("-" * 15)

    while True:
        action = GET_USER_RESPONSE("Enter flight number (1, 2, etc.): ")

        # --- START TRY/EXCEPT for Converting Input to Number ---
        try:
            choice_id = int(action)
            chosen_flight = next((f for f in available_flights if f['ID'] == choice_id), None)

            if chosen_flight:
                # 1. Check risks
                risk_result = apply_risk_check(chosen_flight)

                if risk_result is True:
                    # return True because the game ended during risk checks
                    return True

                if risk_result == "CANCELLED":
                    # return False to restart the turn, as the flight was canceled
                    return False

                # 2. Execute the flight
                return _execute_flight(chosen_flight)
            else:
                print("❌ Invalid flight number.")
        except ValueError:
            # This handles cases where the user enters text instead of a number
            print("❌ Invalid input. Please enter a number.")
        # --- END TRY/EXCEPT ---


def run_game():
    """The main game loop function."""
    initialize_game()

    while not check_game_over():
        minutes_left = MAX_TIME_MINUTES - total_time_minutes
        print(f"\nLocation: {airports[current_location_icao]['Name']} | Health: {current_health:.2f} HP | Time Left: {minutes_left} min")

        # If handle_player_turn returns True, the game is over
        if handle_player_turn():
            break

    print("\n--- END OF MISSION ---")


if __name__ == "__main__":
    try:
        while True:
            run_game()

            play_again = GET_USER_RESPONSE("Play another mission? (Yes/No): ")

            if play_again != 'YES':
                break
    except KeyboardInterrupt:
        print("\nGame closed.")
        pass


Database Error occurred: 2005 (HY000): Unknown MySQL server host 'mysql.metropolia.fi' (11001)

------ BEGIN MISSION (75.0 HP) ------
STARTING AT: London Heathrow.
GOAL: Reach Hamad Intl.

Location: London Heathrow | Health: 75.00 HP | Time Left: 1440 min

--- YOUR MOVE ---
  [1] Fly to Hamad Intl (Loss: 20.00 HP)
---------------

⚠️ EMERGENCY: Engine Trouble!
PENALTY: Time +90 min, Health -6.00 HP.

Game closed.
